
# Intro to Pandas, Data Manipulation and Visualization in Python
In this section, we will learn and practice how to read in data, conduct data manipulation and visualization in `Python`. In particular, we will be learning the `Pandas` package, which provides a fast and powerful interface to dataframes. 

## Pandas 
<img src="https://c402277.ssl.cf1.rackcdn.com/photos/13100/images/featured_story/BIC_128.png?1485963152" align="right">
`Pandas` is a library that provides high-performance, easy-to-use data structures and data analysis tools for `Python`.



Let's load the package `pandas` as well as `numpy`, and `matplotlib` for visualization later. The next few parameters set up the inline plotting to look nicely for the notebook. This is standard preamble for data processing in `ipython` notebooks that you can use in the future. There are some other variations such as giving `matplotlib` the `ggplot` theme from `R` if you wish (add `plt.style.use('ggplot')`). 




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

## Case Study
<img  width="200" height="40" src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Instacart_logo_and_wordmark.svg/1280px-Instacart_logo_and_wordmark.svg.png" align="left">
<br>
### Instacart orders


In this problem, we'll use the dataset from Instacart.com (https://www.instacart.com/datasets/grocery-shopping-2017), a grocery delivery service that connects customers with Personal Shoppers who pick up and deliver the groceries from local stores. The open data contains order, product, and aisles detailed information. We took a 5% sample of orders in this tutorial.



### Read in Data
Now let's read in a csv file for the dataset `orders.csv` and `orders_products.csv` using the `read_csv` function in pd. Index is very important in Pandas for reasons we will talk about later (subset, merge, ...). Let's specify the index when we read in the data with `index_col = ` parameter. 

To get a glimpse of the data, you can do:
* `.shape` to look at the dimension / size / shape of the dataframe,
* `.describe()` to see a summary of the data,
* `.head()` to view first 5 rows, or you can do it with `[:5]`.

In [ ]:
orders = pd.read_csv('data/orders.csv', index_col='order_id');
print(orders.shape)
print(orders.describe())
orders.head()

In [ ]:
orders_products = pd.read_csv('data/orders_products.csv', index_col='order_id');
orders_products.head()

### Data Indexing

After reading in the datasets and taking a look at the description or the first few rows, we are interested in some basic dataframe manipulations. 
* **Subset columns:** To select a column, we can:
    1. index with the name of the column as a string, 
    2. use the attribute operator . on the column name,
    3. use the `loc[:, ]` function on the column name,
    4. use the `iloc[:, ]` function on the column index (remember zero indexing in Python!)

In this example we take the `order_hour_of_day` column using each of the method.

In [ ]:
print(orders['order_hour_of_day'].head())
print(orders.order_hour_of_day.head())
print(orders.loc[:, 'order_hour_of_day'].head())
print(orders.iloc[:, 4].head())

You can also select multiple columns by indexing the list of columns you would like to select:

In [ ]:
orders[['order_dow', 'order_hour_of_day']].head()

* **Subset rows**: You can subset the rows of a dataframe by 
    1. `iloc[]`: based on the row numbers
    2. `loc[]`: based on index value
    3. `[]` with a logical condition

Let's look at the following examples:

To get the first 5 rows, use `iloc`:

In [ ]:
orders.iloc[range(5)]

To subset based on the index value, use the `loc` command:

In [ ]:
orders.loc[[1076138,1609528]]

If we only want to look at the order hour of day being 6pm:

In [ ]:
orders_18pm = orders[orders['order_hour_of_day'] == 18]
print(orders_18pm.shape)
orders_18pm[:5]

Another example use case could be that if we want to only keep observations with non-NA values for `days_since_prior_order`, we can use the `isnull()` function which returns a boolean array for indexing:

In [ ]:
orders_noNA = orders[~orders['days_since_prior_order'].isnull()];
orders_noNA.head()

## Exercise 1: Explore `aisles` and `products` data
Please read in the `aisles.csv` and `products.csv` files. Answer the following questions:
* How many distinct aisles are there?
* What's the name for aisle_id = 61? 
* How many products are there in aisle_id = 61?


### Visualization

Let's try to look at the relationship between the day-of-the-week and the hour-of-day for all orders. We can tabulate them by using the `crosstab()` function:

In [ ]:
orders_counts = pd.crosstab(orders['order_hour_of_day'], orders['order_dow'])
orders_counts

One interesting way to look at this data is to plot the distribution of hour-of-day by different day-of-the-week. A `Pandas` dataframe has some plot functions that can be called directly on it. For example, to do a line plot of the counts by each, 

In [ ]:
orders_counts.plot()

That's pretty good already, without us needing to supply any arguments to the plot function. Monday and Sundays seem to be the days with more orders placed than other days. It also seems like there is a little bump on Monday, at around 9 to 10am.

The lines for the other days seem to be overlapping and a bit hard to discern, so maybe it's a good idea to separate out the lines in multiple plots (think about the `facet` in `ggplot`). Try the following, which groups the counts by the two variables, count the sizes, and gives a plot separated by the first variable:

In [ ]:
orders.groupby(['order_dow','order_hour_of_day']).size().plot()

This is useful, but the labels seem a bit off. To slightly improve on this, we can do a loop of subplots:

In [ ]:
fig, axarr = plt.subplots(1,7, sharey=True)
for i in range(7):
    df = orders[orders['order_dow']==i].groupby('order_hour_of_day').size()
    df.index.name = 'hour'
    df.plot(ax=axarr[i], title='Day ' + str(i))

fig.tight_layout()

Another question that might be interesting is to look at the median, quantile, etc. of the hour of day. We can use a `boxplot()` function:

In [ ]:
orders.boxplot(column='order_hour_of_day', by='order_dow')

Maybe a heatmap could also be useful? To make a heatmap, we can use the `pcolor()` function. We provide the data and the color mapping as arguments to the function:

In [ ]:
plt.pcolor(orders_counts, cmap=plt.cm.Blues)
plt.colorbar()

## Exercise 2: visualize the relationship between `days_since_prior_order` and `order_day_of_week`

We are interested in looking at the `days_since_prior_order` variable and the `order_day_of_week` variable for the orders that are placed between 9am to 5pm. 

## Group, Summarize, and Sort

Suppose we are intereted in knowing something on the individual user level. For example, what's the total number of orders each user had? We can use the `groupby` and `size`. For a single variable this achieves similar effect as `value_counts()` function on that column:

In [ ]:
orders.groupby('user_id').size()[:5]

In [ ]:
orders['user_id'].value_counts().head()

We can sort it further by the `sort_values()` function (and specifying `ascending = False` for decreasing order):

In [ ]:
orders.groupby('user_id').size().sort_values(ascending = False)[:5]

More generally, you can use the `agg` for aggregating specific summary statistic. 
* You can supply a single type and it will be performed on all variables: for example, getting the mean of each variable on each user:

In [ ]:
orders.groupby('user_id').agg('mean')[:5]

* or supply a dictionary that the specfic variable as key: here only summarizses the mean of the `order_hour_of_day`, and the maximum of the `days_since_prior_order`:

In [ ]:
orders.groupby('user_id').agg({'order_hour_of_day': 'mean', 'days_since_prior_order': 'max'})[:5]

## Exercise 3: Describing User Patterns

From the `orders_products` dataframe, answer the following questions: 
* Are there more products that are reordered, or never ordered again?
* Which `product_id` is the most frequently ordered? 
* What is that product called from the `products` dataframe?
* Is there a relationship between the order when a product is added to cart (`add_to_cart_order`), and whether a product is reordered? 
* What about the relationship between whether the product is reordered and the total number of items in the order?

## Merge and Join

With many separate dataframes, it is often useful to join them to understand the relationship between variables and also to create additional features in predictions. Pandas provides high-performance, in memory join operations that are similar to relational databases such as SQL.

When the two dataframes are both indexed by the same variable that you want to join on, it is easy: use the `join` function on the left dataframe, and the right data frame is supplied as second argument.

If the current index is not the right variable, we can first reindex the data using the `set_index` command:
```python
orders_products=orders_products.set_index('order_id')
orders=orders.set_index('order_id')
```

In [ ]:
orders_products_joined = orders_products.join(orders);
orders_products_joined.head()

When the two columns do not have the same index, we can use the more flexible `merge` function:
* the `left_on` and `right_on` options specify the column names to be joined on
* if the variable to be joined on is an index, set `left_index` / `right_index` to `True`. 
* Finally, the `how` option allows you to specify the type of joins (left, right, inner, outer).

In [ ]:
order_products_desc = orders_products.merge(products, left_on='product_id', right_index=True, how='left')
order_products_desc.head()

This way, we can look at the most popular products.

In [ ]:
order_products_desc['product_name'].value_counts()[:10].plot(kind='bar')

## Exercise 4: Summarizing orders by aisle information

We are interested in knowing which aisles are *LEAST* popular and can make management decisions based on that. Try to answer the following questions:
* Which aisle has the least number of products ordered from?
* What about only among the reordered products?


**Challenge:** Suppose there was a software glitch that all products with the `add_to_cart_order` more than 30 was not correctly charged. What percent of all orders are affected by this glitch?

## Reading data from SQL databases

(Note: The content for this section is adapted from the Pandas Cookbook Chapter 9.)

Pandas can read from HTML, JSON, SQL, Excel, HDF5, Stata, and a few other things. We'll talk about reading data from SQL databases now.

You can read data from a SQL database using the `pd.read_sql` function. `read_sql` will automatically convert SQL column names to DataFrame column names.

`read_sql` takes 2 arguments: a `SELECT` statement, and a database connection object. It means you can read from *any* kind of SQL database -- it doesn't matter if it's MySQL, SQLite, PostgreSQL, or something else.

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
con = sqlite3.connect("data/weather_2012.sqlite")
df = pd.read_sql("SELECT * from weather_2012 LIMIT 5", con)
df

And that's it! If you are familiar with SQL type statements, you can try some advanced `SELECT` statements; otherwise, just select everything you need and do the data cleaning in Pandas. 

## Getting Data Ready for SciKitLearn 

Having a cleaned `Pandas` dataframe does not allow you to run through machine learning packages directly yet. The dataframe may need to be appropriately transformed (onehot encoded for categorical variables, scaled, etc.). Both Pandas and scikit-learn offer some useful preprocessing functions.

`pd.get_dummies` takes the dataframe, and a list of categorical columns to be converted into a dummified dataframe. See the following example, where we take the `orders_products_aisles` dataframe that we merged earlier, and convert the string categorical variable `aisle` to be onehot encoded:

In [ ]:
X_dum = pd.get_dummies(orders_products_aisles.drop('reordered', axis=1) , columns=['aisle'])
X_dum.head()

Now we can run it through sklearn, which will be the topic of tomorrow's lesson.

In [ ]:
from sklearn import tree
Y = order_products_desc['reordered']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_dum, Y)
clf.predict_proba(X_dum)[:,1]

## Other useful references

* A useful cheatsheet: https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf
* The mapping between `R` commands and `Pandas` can be found here, if you are coming from a more `R`-type background:
https://pandas.pydata.org/pandas-docs/version/0.18.1/comparison_with_r.html

<img src="http://www.rantchic.com/wp-content/uploads/2014/10/panda.jpg" align="left">